<a href="https://colab.research.google.com/github/RaghavMaskara21/Machine-Learning/blob/main/8_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
from numpy.random import RandomState
import pandas as pd 

import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [ ]:
data = pd.read_csv("spam.csv", encoding = "latin-1")
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
train_csv1 = data[0:2500]
test_csv = data[2500:] 

rng = RandomState()

train_csv = train_csv1.sample(frac=0.7, random_state=rng)
dev_csv = train_csv1.loc[~train_csv1.index.isin(train_csv.index)]

print(train_csv.shape, test_csv.shape, dev_csv.shape)
train_csv.head(3)

(1750, 2) (1215, 2) (750, 2)


,v1,v2
789,ham,Gud mrng dear hav a nice day
818,ham,Then why you not responding
1858,ham,"Sir, i am waiting for your call."


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_csv.v1.append(test_csv.v1.append(dev_csv.v1)))
train_csv['ENCODED_INT'] = le.transform(train_csv.v1)
dev_csv['ENCODED_INT'] = le.transform(dev_csv.v1)
test_csv['ENCODED_INT'] = le.transform(test_csv.v1)

train_csv.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,v1,v2,ENCODED_INT
789,ham,Gud mrng dear hav a nice day,0
818,ham,Then why you not responding,0


In [ ]:
train_csv.v1.value_counts()

ham     1511
spam     239
Name: v1, dtype: int64

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Tf-Idf

In [ ]:
train_csv.head()

,v1,v2,ENCODED_INT
789,ham,Gud mrng dear hav a nice day,0
818,ham,Then why you not responding,0
1858,ham,"Sir, i am waiting for your call.",0
2261,ham,Did you show him and wot did he say or could u...,0
1846,ham,Its on in engalnd! But telly has decided it wo...,0


In [ ]:
dev_csv.head()

,v1,v2,ENCODED_INT
0,ham,"Go until jurong point, crazy.. Available only ...",0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
20,ham,Is that seriously how you spell his name?,0
22,ham,So Ì_ pay first lar... Then when is da stock c...,0


In [ ]:
train_X = []
def train_cleaner(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

val_X = []
def val_cleaner(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    val_X.append(review)

test_X = []
def test_cleaner(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

train_csv.v2.apply(train_cleaner)
dev_csv.v2.apply(val_cleaner)
test_csv.v2.apply(test_cleaner)


2500    None
2501    None
2502    None
2503    None
2504    None
        ... 
3710    None
3711    None
3712    None
3713    None
3714    None
Name: v2, Length: 1215, dtype: object

In [ ]:
train_y = train_csv.ENCODED_INT.values
test_y = test_csv.ENCODED_INT.values
val_y = dev_csv.ENCODED_INT.values
print(val_y)

[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1
 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 1 0 0 0 0 1 0 0 0 1 0 

In [ ]:
tf_idf = TfidfVectorizer()

X_train_tf = tf_idf.fit_transform(train_X)

X_train_tf = tf_idf.transform(train_X)
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

X_test_tf = tf_idf.transform(test_X)
print("n_samples: %d, n_features: %d" % X_test_tf.shape)


n_samples: 1750, n_features: 3795
n_samples: 1215, n_features: 3795


In [ ]:
print(X_train_tf[0])

  (0, 2188)	0.4042861859567367
  (0, 2112)	0.5183327076164543
  (0, 1424)	0.44428979532929513
  (0, 1375)	0.4082435712021524
  (0, 783)	0.3428216465765709
  (0, 777)	0.2937702623419278


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_tf, train_y)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred = clf.predict(X_test_tf)


In [ ]:
print(classification_report(le.inverse_transform(test_y), le.inverse_transform(y_pred)))


              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1073
        spam       1.00      0.79      0.88       142

    accuracy                           0.98      1215
   macro avg       0.99      0.89      0.93      1215
weighted avg       0.98      0.98      0.97      1215



## CNN

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils



In [ ]:
t = Tokenizer(oov_token='<UNK>')

t.fit_on_texts(train_csv.v2)
t.word_index['<PAD>'] = 0


In [ ]:
max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']

(('slippers', 4782), ('<PAD>', 0), 1)

In [ ]:
train_sequences = t.texts_to_sequences(train_csv.v2)
val_sequences = t.texts_to_sequences(dev_csv.v2)
test_sequences = t.texts_to_sequences(test_csv.v2)


In [ ]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=4783
Number of Documents=1750


In [ ]:
MAX_SEQUENCE_LENGTH = 25
EMBED_SIZE = 300
EPOCHS=5
BATCH_SIZE=128
VOCAB_SIZE = len(t.word_index)

X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_val = sequence.pad_sequences(val_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_val.shape, X_test.shape


((1750, 25), (750, 25), (1215, 25))

In [ ]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 25, 300)           1434900   
                                                                 
 conv1d_6 (Conv1D)           (None, 25, 128)           153728    
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 12, 128)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 12, 64)            32832     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 6, 64)            0         
 1D)                                                             
                                                                 
 conv1d_8 (Conv1D)           (None, 6, 32)            

In [ ]:
train_y_oh = np_utils.to_categorical(train_y)
val_y_oh = np_utils.to_categorical(val_y)
 
print(X_train.shape)
print(train_y_oh.shape)
print(X_val.shape)
print(val_y_oh.shape)

(1750, 25)
(1750, 2)
(750, 25)
(750, 2)


In [ ]:
model.fit(X_train, train_y_oh, 
          validation_data=(X_val, val_y_oh),
          epochs=1, 
          batch_size=BATCH_SIZE, 
          workers=8,
          verbose=1)

14/14 [==============================] - 3s 129ms/step - loss: 0.4205 - accuracy: 0.8571 - val_loss: 0.2253 - val_accuracy: 0.8587


In [ ]:
y_pred = model.predict(X_test)
y_pred.shape


(1215, 2)

In [ ]:
y_pred = np.argmax(y_pred, axis=-1)


In [ ]:
print(classification_report(le.inverse_transform(test_y), le.inverse_transform(y_pred)))


              precision    recall  f1-score   support

         ham       0.88      1.00      0.94      1073
        spam       0.00      0.00      0.00       142

    accuracy                           0.88      1215
   macro avg       0.44      0.50      0.47      1215
weighted avg       0.78      0.88      0.83      1215



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
